In [2]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 5.6 MB/s eta 0:00:00


In [3]:
from sentence_transformers import SentenceTransformer, util
import json
import torch
import tqdm

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
small_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
large_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
def load_json(file_name):
    with open(file_name, 'r') as f:
        return json.load(f)

In [7]:
corpus = load_json("/content/drive/MyDrive/corpus.json")
queries = load_json("/content/drive/MyDrive/queries.json")
qrels = load_json("/content/drive/MyDrive/qrels.json")

In [8]:
corpus_texts = list(corpus.values())

In [9]:
small_corpus_embeddings = small_model.encode(corpus_texts, convert_to_tensor=True)

KeyboardInterrupt: 

In [15]:
corpus_ids = list(corpus.keys())

In [18]:
large_corpus_embeddings = large_model.encode(corpus_texts, convert_to_tensor=True)

In [22]:
torch.save(large_corpus_embeddings, 'large_corpus_embeddings.pt')

In [22]:
torch.save(small_corpus_embeddings, 'small_corpus_embeddings.pt')

In [10]:
large_corpus_embeddings = torch.load('/content/drive/MyDrive/large_corpus_embeddings.pt',map_location=torch.device('cpu'))
small_corpus_embeddings = torch.load('/content/drive/MyDrive/small_corpus_embeddings.pt',map_location=torch.device('cpu'))

<ipython-input-10-9cd44ed1c7a1>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  large_corpus_embeddings = torch.load('/content/drive/MyDrive/large_corpus_embeddings.pt',map

In [11]:
query_id = list(queries.keys())[0]
query = queries[query_id]
top_k = 10

In [12]:
def retrieve_candidates(query, model, corpus_embeddings, top_k=10):
    query_embedding = model.encode(query, convert_to_tensor=True)
    scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)
    top_results = torch.topk(scores, k=top_k)
    return top_results

In [13]:
top_results_small = retrieve_candidates(query, small_model, small_corpus_embeddings, top_k=top_k)

In [16]:
print(f"Top 10 candidates for query '{query}':")
for score, idx in zip(top_results_small[0][0], top_results_small[1][0]):
    print(f"Document: {corpus[corpus_ids[idx]]['text']} (Score: {score.item()})")

Top 10 candidates for query 'How to deposit a cheque issued to an associate in my business into my business account?':
Document: Just have the associate sign the back and then deposit it.  It's called a third party cheque and is perfectly legal.  I wouldn't be surprised if it has a longer hold period and, as always, you don't get the money if the cheque doesn't clear. Now, you may have problems if it's a large amount or you're not very well known at the bank.  In that case you can have the associate go to the bank and endorse it in front of the teller with some ID.  You don't even technically have to be there.  Anybody can deposit money to your account if they have the account number. He could also just deposit it in his account and write a cheque to the business. (Score: 0.7807607650756836)
Document: "I have checked with Bank of America, and they say the ONLY way to cash (or deposit, or otherwise get access to the funds represented by a check made out to my business) is to open a busi

In [29]:
top_results_large = retrieve_candidates(query, large_model, large_corpus_embeddings, top_k=top_k)

In [30]:
print(f"Top 10 candidates for query '{query}':")
for score, idx in zip(top_results_large[0][0], top_results_large[1][0]):
    print(f"Document: {corpus[corpus_ids[idx]]['text']} (Score: {score.item()})")

Top 10 candidates for query 'How to deposit a cheque issued to an associate in my business into my business account?':
Document: Just have the associate sign the back and then deposit it.  It's called a third party cheque and is perfectly legal.  I wouldn't be surprised if it has a longer hold period and, as always, you don't get the money if the cheque doesn't clear. Now, you may have problems if it's a large amount or you're not very well known at the bank.  In that case you can have the associate go to the bank and endorse it in front of the teller with some ID.  You don't even technically have to be there.  Anybody can deposit money to your account if they have the account number. He could also just deposit it in his account and write a cheque to the business. (Score: 0.7576274275779724)
Document: Assuming it's your business, endorse the check as yourself and your DBA name, payable to your personal account (Score: 0.63334059715271)
Document: "If the cheque is not crossed, then your

In [31]:
def save_top_results(file_name, top_results):
    with open(file_name, 'w') as f:
        json.dump(top_results, f)

In [32]:
top_k_results = {}
for query_id, query_text in queries.items():
    top_results = retrieve_candidates(query_text, large_model, large_corpus_embeddings, top_k=10)

In [33]:
top_k_results[query_id] = {
        'query': query_text,
        'top_k_docs': [corpus_ids[idx.item()] for idx in top_results[1][0]],
        'scores': [score.item() for score in top_results[0][0]]
    }

In [34]:
save_top_results("top_k_results.json", top_k_results)